In [59]:
import csv
import numpy as np
import pandas as pd


class Day:
    def __init__(self, use):
        self.season = 'summer'
        self.weekday = 'weekday'
        self.use = use
        self.dayuse = sum(self.use)
        
    def setSeason(self, season):
        self.season = season
    
    def setDay(self, weekday):
        self.weekday = weekday

In [60]:
customersumm = pd.read_csv('anonymized_1in10_actual_actual_2014_cluster_summary.csv', index_col=1, header=0)

filename = 'pge-res-PGFG-res_misc-noKW-nonCare-0.0_0.1' #change to appropriate filename

customers = int(customersumm.loc[filename, 'customer_count'])

df = pd.read_csv(filename+'.csv', index_col=0)
df['total'] = df['total']/customers

In [61]:
daylist = []

use = []
for i in range(1, df.shape[0] + 1):
    use.append(float(df.loc[i, 'total']))
    if i%24 == 0:
        daylist.append(Day(use))
        use = []

In [62]:
def flat(daylist, price=0.17):
    totaluse = 0
    for day in daylist:
        totaluse = totaluse + day.dayuse
    return totaluse * price


def tier(daylist, baseline=15, tier1=0.22376, tier2=0.28159, tier3=0.49334):
    # https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_E-1.pdf
    totaluse = 0
    for day in daylist:
        totaluse = totaluse + day.dayuse
    return min(baseline, totaluse) * tier1 + max(0, totaluse - baseline) * tier2 + max(0, totaluse - 4*baseline) * tier3


def tou(daylist, speak=0.25354, soffpeak=0.20657, wpeak=0.18022, woffpeak=0.17133):
    # https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_EL-TOU.pdf
    speaksum = 0
    soffpeaksum = 0
    wpeaksum = 0
    woffpeaksum = 0
    for day in daylist:
        if day.season == "summer":
            if day.weekday == "weekday":
                speaksum = speaksum + sum(day.use[15:20])
                soffpeaksum = soffpeaksum + sum(day.use[0:14]) + sum(day.use[21:23])
            else:
                soffpeaksum = soffpeaksum + sum(day.use)
        else:
            if day.weekday == "weekday":
                wpeaksum = wpeaksum + sum(day.use[15:20])
                woffpeaksum = woffpeaksum + sum(day.use[0:14]) + sum(day.use[21:23])
            else:
                woffpeaksum = woffpeaksum + day.dayuse
    return speaksum * speak + soffpeaksum * soffpeak + wpeaksum * wpeak + woffpeaksum * woffpeak


print(flat(daylist))
print(tier(daylist))
print(tou(daylist))

190.09162169605474
836.048034711316
211.5598301073641
